In [ ]:
!pip install -q transformers pytorch-lightning==1.9 wandb ninja #deepspeed

In [ ]:
from shutil import copy
import os

base_model_name = "RWKV_MLDY" #@param {type: "string"}
base_model_path = "models/{base_model_name}*.pth"
tuned_model_name = base_model_name
output_path = os.getcwd()


def training_options():
    EXPRESS_PILE_MODE = False
    EXPRESS_PILE_MODEL_NAME = base_model_path.split(".")[0]
    EXPRESS_PILE_MODEL_TYPE = base_model_name
    datafile = "train_data/midi_data.txt" #@param {type: "string"}
    lr_init = 0.00001 #@param {type: "number"}
    lr_final = lr_init
    n_epoch = 10 #@param {type: "number"}
    epoch_save_frequency = 10 #@param {type: "number"}
    batch_size = 4 #@param {type: "number"}
    n_layer = 12 #@param {type: "number"}
    n_embd = 768 #@param {type: "number"}
    ctx_len = 2048 #@param {type: "number"}
    LOAD_MODEL = False # @param {type:"boolean"}
    EPOCH_BEGIN = 0 #@param {type: "number"}
    NUM_GPUS = 1 #@param {type: "number"}
    epoch_save_path = f"{output_path}/{tuned_model_name}"

    return locals()

def model_options():
    T_MAX = 2048 #@param {type: "number"}
    return locals()

def env_vars():
    RWKV_FLOAT_MODE = 'fp16' # ['fp16', 'bf16', 'bf32'] {type:"string"}
    RWKV_DEEPSPEED = '0' # ['0', '1'] {type:"string"}
    return {f"os.environ['{key}']": value for key, value in locals().items()}

def replace_lines(file_name, to_replace):
    with open(file_name, 'r') as f:
        lines = f.readlines()
    with open(f'{file_name}.tmp', 'w') as f:
        for line in lines:
            key = line.split(" =")[0]
            if key.strip() in to_replace:
                value = to_replace[key.strip()]
                if isinstance(value, str):
                    f.write(f'{key} = "{value}"\n')
                else:
                    f.write(f'{key} = {value}\n')
            else:
                f.write(line)
    copy(f'{file_name}.tmp', file_name)
    os.remove(f'{file_name}.tmp')

values = training_options()
values.update(env_vars())
replace_lines('train.py', values)
replace_lines('src/model.py', model_options())

In [ ]:
!python train.py

In [ ]:
!python run.py \
    --model_name checkpoints/RWKV_MLDY1 \
    --num_trials 1 \
    --length_per_trial 2048 \
    --temperature 1.0\
    --top_p 0.90 \
    --top_p_newline 0.94